In [1]:
import pandas as pd
import utility

pricesDataFolder = "prices_data/"
dfH4 = pd.read_csv(
    filepath_or_buffer=pricesDataFolder+"H4/[SP500]_H4_2014-03-20__2024-03-20.csv",
    delimiter="\t"
)

dfH4 = dfH4.rename(
    columns = {
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low', 
        '<CLOSE>':'close'
    }
)

dfH4['datetime'] = pd.to_datetime(dfH4['<DATE>'] + ' ' + dfH4['<TIME>'])
dfH4 = dfH4.drop(['<TICKVOL>', '<VOL>', '<SPREAD>', '<DATE>', '<TIME>'], axis=1)
dfH4 = utility.heikinashi(dfH4) #add heikin ashi candles
dfH4["shortTermMA"] = dfH4["close"].rolling(window=50).mean() # add moving average 50
dfH4["longTermMA"] = dfH4["close"].rolling(window=200).mean() # add moving average 200
dfH4 = utility.addHACandleColor(dfH4)

In [2]:

from actor_critic_training import actorCritictrainingLoop
from actor_critic.trading_agent.agent import Agent
from strategies.Heikin_Ashi_Moving_Average_Strategy import HeikinAshiMovingAverage

agent = Agent()
strategy = HeikinAshiMovingAverage(agent, useSR="SRbyRL", useUpdateSl=True, uselongTermMA=False)

actorCritictrainingLoop(dfH4, strategy)

2024-03-31 21:33:56.629309: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 21:33:57.468791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-31 21:33:58.908474: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-31 21:33:58.974404: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required li

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 3 values, but the requested shape has 4 [Op:Reshape]